**Name:** Wil Aquino

**Date:** October 22, 2021

**Project:** Individual Assignment 2

**Note:** This code was adapted from the Week 6 HW.

In [1]:
!pip install pandas
#!pip uninstall pandas-datareader
!pip install anvil-uplink --upgrade

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
import numpy as np
import pandas as pd

In [3]:
def clean_data(original_data):
    """ Cleans vaccination tweets data.
    
    Args:
        original_data::[pd.DataFrame]]
            The vaccination tweets data to clean.
            
    Return:
        cleaner_data::[pd.DataFrame]
            The cleaned data in the form of a Pandas DataFrame.
    """
    
    years = original_data['date'].str.extract(r'([0-9]{4})')
    years_in_2020 = years[0] == '2020'
    data_in_2020 = original_data[years_in_2020]
    
    useless_columns = ['id', 'user_location', 'user_description', 'user_created', 'hashtags', 'source', 'is_retweet']
    cleaned_data = data_in_2020.drop(columns = useless_columns)
    
    cleaner_data = cleaned_data.replace(regex = r'[^ -~]+', value = '')
    return cleaner_data

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def retrieve_sentiment_analysis(df):
    """ Performs a sentiment analysis on the given tweets.
    
    Args:
        df::[pd.DataFrame]
            The table of given tweets and their statistics.
            
    Return:
        tweets_df::[pd.DataFrame]
            The tweets with their sentiment analysis data.
    """
    
    analyzer = SentimentIntensityAnalyzer()
    tweets = {
        "tweet": [],
        "negative": [],
        "neutral": [],
        "positive": []
    }

    for sentence in df['text']:
        vs = analyzer.polarity_scores(sentence)

        tweets["tweet"].append(sentence)
        tweets["negative"].append(vs["neg"])
        tweets["neutral"].append(vs["neu"])
        tweets["positive"].append(vs["pos"])
        
    tweets_df = pd.DataFrame(tweets)
    
    total = len(tweets_df)
    negative_avg = sum(tweets["negative"]) / total
    neutral_avg = sum(tweets["neutral"]) / total
    positive_avg = sum(tweets["positive"]) / total

    print(f"\nAverage negative tweets: {negative_avg * 100}%\n")
    print(f"Average neutral tweets: {neutral_avg * 100}%\n")
    print(f"Average positive tweets: {positive_avg * 100}%\n")
    
    return tweets_df

In [5]:
# test
f = open('vaccination_tweets.csv', encoding="utf8")
df = pd.read_csv(f)
f.close()

data = clean_data(df)
sentiment_analysis = retrieve_sentiment_analysis(data)
sentiment_analysis


Average negative tweets: 3.519242424242424%

Average neutral tweets: 87.6289898989899%

Average positive tweets: 8.851868686868674%



,tweet,negative,neutral,positive
0,Same folks said daikon paste could treat a cyt...,0.000,0.803,0.197
1,While the world has been on the wrong side of ...,0.125,0.766,0.109
2,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,0.000,0.875,0.125
3,"Facts are immutable, Senator, even when you're...",0.000,1.000,0.000
4,Explain to me again why we need a vaccine @Bor...,0.000,1.000,0.000
...,...,...,...,...
1975,"On #vaccines, .@realDonaldTrump did everythin...",0.000,1.000,0.000
1976,This opens the way for regulators worldwide to...,0.000,1.000,0.000
1977,First dose of #PfizerBioNTech vaccine received...,0.000,1.000,0.000
1978,To vax or not to vax?Check out my latest artic...,0.000,1.000,0.000


In [6]:
import anvil.server
import anvil.media
import anvil.tables as tables
from anvil.tables import app_tables
from io import StringIO

# connect to Anvil server
anvil.server.connect("server_UMLVNXYSCYB5FTL7RURJ7XJH-RWDYPUOYYSODI4VZ")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
@anvil.server.callable
def pull_data():
    return app_tables.import_data.get(name='original_data')['data_file']

@anvil.server.callable
def convert_media_to_df(media):
    my_bytes = file.get_bytes()
    my_string = str(my_bytes, 'utf-8')
    return pd.read_csv(StringIO(my_string))

@anvil.server.callable
def get_original_data():
    return app_tables.original_data.search()
  
@anvil.server.callable
def import_original_data_to_anvil(data):
    global original_df
    original_df = data.replace({np.nan: None})
    
    for d in original_df.head(12).to_dict(orient="records"):
        app_tables.original_data.add_row(**d)
        
anvil.server.wait_forever()

Anvil websocket closed (code 1006, reason=Going away)


Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 398, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 390, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\_threaded_server.py", line 412, in do_call
    raise error_from_server
anvil._server.AnvilWrappedError: 'Connection to Anvil Uplink server lost'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Wil\anaconda3\envs\data-x\l

Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


Exception in thread Thread-21:
Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 398, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 390, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\_threaded_server.py", line 412, in do_call
    raise error_from_server
anvil._server.AnvilWrappedError: 'Connection to Anvil Uplink server lost'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Wil\anaconda3\envs\data-x\l

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


Exception in thread Thread-24:
Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 398, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 390, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\_threaded_server.py", line 412, in do_call
    raise error_from_server
anvil._server.AnvilWrappedError: 'Connection to Anvil Uplink server lost'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Wil\anaconda3\envs\data-x\l

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


Exception in thread Thread-27:
Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 398, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\server.py", line 390, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\site-packages\anvil\_threaded_server.py", line 412, in do_call
    raise error_from_server
anvil._server.AnvilWrappedError: 'Connection to Anvil Uplink server lost'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Wil\anaconda3\envs\data-x\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Wil\anaconda3\envs\data-x\l

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
